In [ ]:
# Imports
import pandas as pd
import numpy as np
import duckdb

In [ ]:
# Load dnc file and raw file
dnc_df = duckdb.read_parquet('output/DNC/dnc_list_final.parquet').df()
raw_df = duckdb.read_csv('dataset/RAW/JAS233_JBT.csv', normalize_names=True).df()

In [ ]:
# Convert phone to int
raw_df['work_phone'] = raw_df['work_phone'].astype(np.int64)

In [ ]:
# Checks if Phone from RAW file is in DNC list
is_dnc = duckdb.sql("""
    SELECT DISTINCT a.*,
        CASE
            WHEN b.value IS NOT NULL THEN 'Yes' ELSE 'No'
        END AS is_dnc
    FROM raw_df a
    LEFT JOIN dnc_df b on a.work_phone = b.value
""").df()

In [ ]:
# Export to Excel
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
                  
    COPY is_dnc TO 'output/jas233_jbt_cleaned_is_dnc.xlsx' WITH (FORMAT GDAL, DRIVER 'xlsx')
""")

In [ ]:
# Report Summary

# Counts
total_leads = is_dnc.shape[0]
unique = is_dnc["work_phone"].nunique()
dupes = total_leads - unique
dnc_count = is_dnc["is_dnc"].value_counts().get("Yes", 0)
good_leads = is_dnc["is_dnc"].value_counts().get("No", 0)

# Percentages, 2 decimal places
total_leads_percentage = f"{round((total_leads / total_leads) * 100, 2)}%"
unique_percentage = f"{round((unique / total_leads) * 100, 2)}%"
dupes_percentage = f"{round((dupes / total_leads) * 100, 2)}%"
dnc_count_percentage = f"{round((dnc_count / total_leads) * 100, 2)}%"
good_leads_percentage = f"{round((good_leads / total_leads) * 100, 2)}%"

# List of lists
report = [["Total Leads", total_leads, total_leads_percentage], 
          ['Unique', unique, unique_percentage],
          ['Duplicates', dupes, dupes_percentage],
          ['DNC', dnc_count, dnc_count_percentage],
          ['Good Leads', good_leads, good_leads_percentage]]

# Create df
report_df = pd.DataFrame(report, columns=['Description', 'Count', 'Percentage'])
report_df

In [ ]:
# Export report
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
                  
    COPY report_df TO 'output/jas233_jbt_report.xlsx' WITH (FORMAT GDAL, DRIVER 'xlsx')
""")